In [ ]:
import pandas as pd
import os
import json
from globals import BASE_DIR

include_categories = True

dataset = "yelp"

DATASET_DIR = f"{BASE_DIR}{dataset}_dataset/"
#DATASET_DIR = f"/Users/andreaforster/Documents/data_thesis/{dataset}_dataset/"


available_datasets = ["foursquaretky", "yelp", "gowalla", "brightkite", "snowcard"]

In [2]:
def open_big_json(file_path):
    data = []

    # Open the file and read it line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Parse the JSON data and append to the list
            data.append(json.loads(line))

    # Create a DataFrame from the list of records
    df = pd.DataFrame(data)

    return df

In [3]:
def convert_to_unix_timestamp(df, column_name):
    """
    Convert a column of timestamps in a DataFrame to Unix timestamps.

    Args:
        df (pd.DataFrame): The DataFrame containing the timestamp column.
        column_name (str): The name of the column with timestamps in "%Y-%m-%d %H:%M:%S" format.

    Returns:
        pd.DataFrame: The DataFrame with an additional column for Unix timestamps.
    """
    # Convert the column to datetime objects
    df[column_name] = pd.to_datetime(df[column_name], format="mixed")
    
    # Convert datetime objects to Unix timestamps
    df[f'{column_name}'] = df[column_name].apply(lambda x: x.timestamp())
    
    return df

In [4]:
if dataset == "snowcard":
    checkin_df = pd.read_csv(DATASET_DIR+"TSC_EEL_EXPORT.csv", encoding="latin1", sep=";", header=None, names=["timestamp:float", "user_id:token", "category_id:token", "category_name:token_seq", "name:token_seq", "user_type:token_seq"])
    checkin_df["item_id:token"], item_id = pd.factorize(checkin_df["name:token_seq"])
    user_df = checkin_df[["user_id:token", "user_type:token_seq"]].drop_duplicates(subset=["user_id:token"])
    poi_df = checkin_df[["item_id:token", "name:token_seq", "category_id:token", "category_name:token_seq"]].drop_duplicates(subset=["item_id:token"])
    coordinates_df = pd.read_excel(DATASET_DIR+"snowcard_lifts.xlsx")
    coordinates_df[["lat:float", "lon:float"]] = coordinates_df["lat_lon"].str.split(', ', expand=True)
    coordinates_df.drop(columns=["lat_lon"], inplace=True)
    poi_df = pd.merge(poi_df, coordinates_df[['category_name:token_seq', 'lat:float', 'lon:float']], on='category_name:token_seq', how='left')
    checkin_df = checkin_df[["user_id:token", "item_id:token", "timestamp:float"]]

elif dataset == "foursquarenyc" or dataset == "foursquaretky":
    checkin_df = pd.read_csv(DATASET_DIR + "foursquare_data.csv", sep=",")
    checkin_df = checkin_df.drop(columns=["timezoneOffset"])
    checkin_df = checkin_df.rename(columns={"venueId": "item_id:token", "venueCategoryId": "category_id:token", "venueCategory": "category_name:token_seq", "userId": "user_id:token", "utcTimestamp": "timestamp:float", "latitude": "lat:float", "longitude": "lon:float"})
    user_df = checkin_df[["user_id:token"]].drop_duplicates()

    poi_df = checkin_df[["item_id:token", "category_id:token", "category_name:token_seq", "lat:float", "lon:float"]].drop_duplicates(subset=["item_id:token"])
    checkin_df = checkin_df[["user_id:token", "item_id:token", "timestamp:float"]]

elif dataset == "gowalla" or dataset == "brightkite":
    checkin_df = pd.read_csv(DATASET_DIR + f"loc-{dataset}_totalCheckins.txt", sep="\t", header=None, names=['user_id:token', 'timestamp:float', 'lat:float', 'lon:float', 'item_id:token'])
    checkin_df = checkin_df[~checkin_df['item_id:token'].isin(["00000000000000000000000000000000", "ede07eeea22411dda0ef53e233ec57ca"])]
    user_df = pd.read_csv(DATASET_DIR + f"loc-{dataset}_edges.txt", sep="\t", header=None, names=['user_id:token', 'friends:token_seq'])
    user_df = user_df.groupby('user_id:token')['friends:token_seq'].apply(lambda x: ','.join(map(str, x))).reset_index()
    user_df.columns = ['user_id:token', 'friends:token_seq']
    poi_df = checkin_df[['item_id:token', "lat:float", "lon:float"]].drop_duplicates(subset="item_id:token")
    checkin_df = checkin_df.drop(columns=["lat:float", "lon:float"])

elif dataset == "yelp":
    poi_df = pd.read_json(DATASET_DIR + "yelp_academic_dataset_business.json", lines=True)
    poi_df = poi_df.loc[poi_df['is_open'] == 1]
    poi_df = poi_df.drop(columns=["review_count", "stars", "hours", "is_open", "city", "state", "postal_code", "attributes", "address"])
    poi_df = poi_df.rename(columns={"latitude": "lat:float", "longitude": "lon:float", "business_id": "item_id:token", "name":"name:token_seq", "categories":"category_name:token_seq"})
    user_df = open_big_json(DATASET_DIR + "yelp_academic_dataset_user.json")
    user_df = user_df.drop(columns=["review_count", "name", "yelping_since", "useful", "funny", "cool", "elite", "fans", "compliment_hot", "average_stars", "compliment_more", "compliment_profile", "compliment_cute", "compliment_list", "compliment_note", "compliment_plain", "compliment_cool", "compliment_funny", "compliment_writer", "compliment_photos"])
    user_df = user_df.rename(columns={"user_id": "user_id:token", "friends": "friends:token_seq"})
    checkin_df = open_big_json(DATASET_DIR + "yelp_academic_dataset_review.json")
    checkin_df = checkin_df.drop(columns=["text", "cool", "stars", "useful", "funny", "review_id"])
    checkin_df = checkin_df.rename(columns={"user_id": "user_id:token", "business_id": "item_id:token", "date": "timestamp:float"})
    

In [5]:
checkin_df.sort_values(by="timestamp:float", ascending=True, inplace=True)  

In [8]:
poi_df

,item_id:token,name:token_seq,lat:float,lon:float,category_name:token_seq
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,38.551126,-90.335695,"Shipping Centers, Local Services, Notaries, Ma..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,39.955505,-75.155564,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,40.338183,-75.471659,"Brewpubs, Breweries, Food"
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,36.269593,-87.058943,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,38.627695,-90.340465,"Sporting Goods, Fashion, Shoe Stores, Shopping..."
...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,53.468419,-113.492054,"Nail Salons, Beauty & Spas"
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,36.115118,-86.766925,"Pets, Nurseries & Gardening, Pet Stores, Hobby..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,39.908707,-86.065088,"Shopping, Jewelry, Piercing, Toy Stores, Beaut..."
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,38.782351,-89.950558,"Fitness/Exercise Equipment, Eyewear & Optician..."


In [7]:
user_df

,user_id:token,friends:token_seq
0,qVc8ODYU5SZjKXVBgXdI7w,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA..."
1,j14WgRoU_-2ZE1aw1dXrJg,"ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A..."
2,2WnXYQFK0hXEoTxPtV2zvg,"LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA..."
3,SZDeASXq7o05mMNLshsdIA,"enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg..."
4,hA5lMy-EnncsH4JoR-hFGQ,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA..."
...,...,...
1987892,fB3jbHi3m0L2KgGOxBv6uw,None
1987893,68czcr4BxJyMQ9cJBm6C7Q,None
1987894,1x3KMskYxOuJCjRz70xOqQ,None
1987895,ulfGl4tdbrH05xKzh5lnog,None


In [6]:
checkin_df

,user_id:token,item_id:token,timestamp:float
4179799,3zBJUlWtPNoZ0uN83ODbyg,2bXm0SynOfxDzfrdrCyXqg,2005-02-16 03:23:22
6295220,3zBJUlWtPNoZ0uN83ODbyg,3g6XqkBikTTbZmTukbeGnw,2005-02-16 03:29:39
4887456,3zBJUlWtPNoZ0uN83ODbyg,PP3BBaVxZLcJU54uP_wL6Q,2005-02-16 04:06:26
1423159,XCsZ3hWa_6oP1WkWvK7pmg,U3grYFIeu6RgAAQgdriHww,2005-03-01 16:57:17
4884223,XCsZ3hWa_6oP1WkWvK7pmg,Aes-0Q_guDeYewMapFs_vg,2005-03-01 16:59:37
...,...,...,...
4878781,lmiiFd9KC15fs4xtEoXRvw,XDMno4l95AXgYOd0yDtHZA,2022-01-19 19:48:13
2710116,2Mb0st9WVyccaz6sKNLHWw,M88FFZZ2o_7QKpCFA_8RtA,2022-01-19 19:48:16
6251618,3TQKP7KlNRdrI2gOkG7slg,jVg-KTXEFIeAq47DTp4Hrw,2022-01-19 19:48:19
5476035,i1PMqye40QWNkJ0MYGHuzg,J0joPXxmN-_9Lzafspqdbw,2022-01-19 19:48:25


In [6]:
# import matplotlib.pyplot as plt
# checkin_df['timestamp'] = pd.to_datetime(checkin_df['timestamp:float'], errors='coerce')

# # Extract the year from the 'timestamp' column
# checkin_df['year'] = checkin_df['timestamp'].dt.year

# # Plot the histogram
# plt.figure(figsize=(10, 6))
# checkin_df['year'].dropna().astype(int).hist(bins=range(int(checkin_df['year'].min()), int(checkin_df['year'].max()) + 1), edgecolor='black')
# plt.title('Histogram of Check-ins by Year')
# plt.xlabel('Year')
# plt.ylabel('Number of Check-ins')
# plt.grid(False)
# plt.show()

In [12]:
if dataset == "yelp":
    checkin_df['timestamp'] = pd.to_datetime(checkin_df['timestamp:float'], errors='coerce')

    # Extract the year from the 'timestamp' column
    checkin_df['year'] = checkin_df['timestamp'].dt.year
    checkin_df = checkin_df[checkin_df['year'] >= 2018]
    checkin_df = checkin_df[checkin_df['year'] < 2020]
    checkin_df.drop(columns=["year", "timestamp"], inplace=True)

In [13]:
checkin_df

,user_id:token,item_id:token,timestamp:float
6664443,vYPvl2ngX8UdDB8Pf0JCPA,THJ0i8yRyx1OfvzLsJXgng,2018-01-01 00:00:11
2542113,PcgyHvdduilIJ-O-z_05Sw,HhZDu-IEC7owaHCeEXSf1g,2018-01-01 00:00:28
4742901,SsL2nYQGx-l_MVOKoJExJw,D8UM3J3mx2cyYPy84yU9ag,2018-01-01 00:00:52
4240067,1T-Y8oA4Frv3eRmGO3ciew,Sf9E4E8yo8actSXHFvrZbQ,2018-01-01 00:02:05
4947842,unekdBHjTsiBVtOX5UwbZg,rDr7zhYBOmC3NzJXcZd1BQ,2018-01-01 00:02:12
...,...,...,...
2636555,Yy5uYBI7PH5fVtfQjYI76Q,l4uH-6afJzbm0NFRp7lKog,2019-12-31 23:58:17
2677248,IlGYj_XAMG3v75rfmtBs_Q,EagkHaaC-kUozD3MPzbRIw,2019-12-31 23:58:18
1252247,FhBHx01UWFh3_R_sphucMA,gWJSE8CNWsHS_sUG_sVoRw,2019-12-31 23:58:52
2728567,U4iKsl_nFscRfNDxNvdZ8Q,xG9oeXDZldT5_CZLLzABsw,2019-12-31 23:59:11


In [14]:
checkin_df_timestamp = checkin_df.copy()

In [15]:
#Step 1: Group by user_id and business_id and count check-ins
checkin_df['checkin_count:float'] = checkin_df.groupby(['user_id:token', 'item_id:token'])['item_id:token'].transform('count')
checkin_df = checkin_df.drop_duplicates(subset=["user_id:token", "item_id:token"], keep="first")


In [16]:
print("Number of users, number of POIs", len(checkin_df["user_id:token"].unique()), len(checkin_df["item_id:token"].unique())
)
print("Sparsity:", 1 - len(checkin_df) / (len(checkin_df["user_id:token"].unique()) * len(checkin_df["item_id:token"].unique())))

Number of users, number of POIs 747847 119726
Sparsity: 0.9999802560021843


In [22]:
def filter_df(df, min_reviews_user=15, min_reviews_business=10):
    while True:

        # Filter users with at least min_reviews reviews
        user_counts = df['user_id:token'].value_counts()
        user_mask = df['user_id:token'].map(user_counts) >= min_reviews_user
        df_filtered = df.loc[user_mask]

        # Filter businesses with at least min_reviews reviews
        business_counts = df_filtered["item_id:token"].value_counts()
        business_mask = df_filtered['item_id:token'].map(business_counts) >= min_reviews_business
        df_filtered = df_filtered.loc[business_mask]

        

        # If the size of the filtered DataFrame didn't change, break the loop
        if df_filtered.shape[0] == df.shape[0]:
            break

        # Update the DataFrame for the next iteration
        df = df_filtered

    return df_filtered

In [23]:
checkin_df_filtered = filter_df(checkin_df, min_reviews_business=10, min_reviews_user=15)

In [24]:
checkin_df_filtered["user_id:token"].value_counts().min()

15

In [25]:
checkin_df_filtered["user_id:token"].nunique()

4150

In [26]:
checkin_df_filtered["user_id:token"].nunique(), checkin_df_filtered["item_id:token"].nunique()

(4150, 5259)

In [27]:
# Step 1: Calculate the value counts of `business_id`
value_counts = checkin_df_filtered['item_id:token'].value_counts().reset_index()
value_counts.columns = ['item_id:token', 'count']

# Step 2: Normalize the counts y dividing by the maximum value count
max_count = value_counts['count'].max()
value_counts['business_popularity:float'] = value_counts['count'] / max_count
 
# Step 3: Merge the normalized counts back into the original DataFrame
checkin_df_filtered = checkin_df_filtered.merge(value_counts[['item_id:token', 'business_popularity:float']], on = "item_id:token", how='left')


In [28]:
def user_popularity_sample_calculator(checkin_df_filtered, poi_df, user_df, sep_num, checkin_df_timestamp):
    # Calculate average popularity per user
    average_popularity_per_user = checkin_df_filtered.groupby('user_id:token')['business_popularity:float'].mean().reset_index()
    average_popularity_per_user.columns = ['user_id:token', 'average_popularity']

    average_popularity_per_user = average_popularity_per_user.sort_values(by="average_popularity", ascending=False)

    
    # Sort by average popularity
    

    # Get top 1000 users
    high_pop_user_df_sample = average_popularity_per_user.head(sep_num)
    
    # Get the middle 1000 users around the median
    median_index = len(average_popularity_per_user) // 2
    start_med_index = max(median_index -int (sep_num/2), 0)
    end_med_index = min(median_index + int(sep_num/2), len(average_popularity_per_user))
    med_pop_user_df_sample = average_popularity_per_user.iloc[start_med_index:end_med_index]
    
    # Get the lowest 1000 users
    low_pop_user_df_sample = average_popularity_per_user.tail(sep_num)

    unique_users = list(set(high_pop_user_df_sample["user_id:token"].tolist() + med_pop_user_df_sample["user_id:token"].tolist() + low_pop_user_df_sample["user_id:token"].tolist()))

    checkin_df_sample = checkin_df_filtered[checkin_df_filtered["user_id:token"].isin(unique_users)]
    checkin_df_sample = checkin_df_sample[checkin_df_sample["user_id:token"].isin(unique_users)]

    # unique_items = checkin_df_sample["item_id:token"].unique()
    # print(len(unique_items))

    user_df_sample = user_df[user_df["user_id:token"].isin(unique_users)]
    poi_df_sample = poi_df[poi_df["item_id:token"].isin(checkin_df_sample["item_id:token"])]

    checkin_df_sample = checkin_df_sample[checkin_df_sample["item_id:token"].isin(poi_df_sample["item_id:token"])]

    checkin_df_timestamp = checkin_df_timestamp[checkin_df_timestamp["user_id:token"].isin(unique_users)]
    checkin_df_timestamp = checkin_df_timestamp[checkin_df_timestamp["item_id:token"].isin(poi_df_sample["item_id:token"])]

    
    



    
    return checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample, checkin_df_timestamp

In [29]:
if checkin_df_filtered["user_id:token"].nunique() > 1500:
    sep_num = 500
else:
    sep_num = checkin_df_filtered["user_id:token"].nunique() // 3

print(sep_num)

500


In [30]:
checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample, checkin_df_timestamp = user_popularity_sample_calculator(checkin_df_filtered, poi_df, user_df, sep_num, checkin_df_timestamp)

In [31]:
checkin_df_sample["item_id:token"].nunique()

4510

In [32]:
checkin_df_sample["user_id:token"].value_counts().min()

10

In [33]:
checkin_df_sample["item_id:token"].value_counts().min()

1

In [34]:
checkin_df_timestamp["item_id:token"].nunique()

4510

In [30]:
def id_factorizer(checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample, checkin_df_timestamp):
    """Overwriting the actual ID with a factorized ID so that we can use the same ID both in RecBole and CAPRI"""
    checkin_df_sample['user_id:token'], user_id_map = pd.factorize(checkin_df_sample['user_id:token'])
    checkin_df_sample['item_id:token'], business_id_map = pd.factorize(checkin_df_sample['item_id:token'])

    # Create mapping dictionaries
    user_id_mapping = {original: i for i, original in enumerate(user_id_map)}
    business_id_mapping = {original: j for j, original in enumerate(business_id_map)}

    high_pop_user_df_sample['user_id:token'] = high_pop_user_df_sample['user_id:token'].map(user_id_mapping)
    med_pop_user_df_sample['user_id:token'] = med_pop_user_df_sample['user_id:token'].map(user_id_mapping)
    low_pop_user_df_sample['user_id:token'] = low_pop_user_df_sample['user_id:token'].map(user_id_mapping)

    checkin_df_timestamp["user_id:token"] = checkin_df_timestamp["user_id:token"].map(user_id_mapping)
    checkin_df_timestamp["item_id:token"] = checkin_df_timestamp["item_id:token"].map(business_id_mapping)

    user_df_sample['user_id:token'] = user_df_sample['user_id:token'].map(user_id_mapping)
    poi_df_sample['item_id:token'] = poi_df_sample['item_id:token'].map(business_id_mapping)


    return checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample, checkin_df_timestamp

In [31]:
checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample, checkin_df_timestamp = id_factorizer(checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample, checkin_df_timestamp)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_58945/4057195747.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_df_sample['user_id:token'] = user_df_sample['user_id:token'].map(user_id_mapping)
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_58945/4057195747.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi_df_sample['item_id:token'] = poi_df_sample['item_id:token'].map(business_id_mapping)


In [32]:
checkin_df_sample["user_id:token"].nunique()

1500

In [33]:
checkin_df_sample

,user_id:token,item_id:token,timestamp:float,checkin_count:float,business_popularity:float
1,0,0,Fri Apr 06 12:52:34 +0000 2012,2,0.061404
2,1,1,Fri Apr 06 12:52:35 +0000 2012,1,0.026316
3,2,2,Fri Apr 06 12:52:49 +0000 2012,33,0.031100
4,3,3,Fri Apr 06 12:53:38 +0000 2012,52,0.039872
5,4,4,Fri Apr 06 12:54:35 +0000 2012,11,0.009569
...,...,...,...,...,...
99680,190,1021,Wed Sep 05 23:15:07 +0000 2012,1,0.012759
99681,126,1877,Wed Sep 05 23:28:47 +0000 2012,1,0.015152
99682,1364,1051,Wed Sep 05 23:30:16 +0000 2012,2,0.018341
99683,100,471,Wed Sep 05 23:32:06 +0000 2012,1,0.028708


In [34]:
def user_id_token_adder(df, column_name_list = ["user_id:token", "item_id:token"]):
    """ Recbole needs a token instead of a number for the user and item ID"""
    for column_name in column_name_list:
        try:
            df[column_name] = df[column_name].astype(str) + "_x"
        except KeyError:
            pass
    return df

In [35]:
checkin_df_sample = user_id_token_adder(checkin_df_sample)
high_pop_user_df_sample = user_id_token_adder(high_pop_user_df_sample)
med_pop_user_df_sample = user_id_token_adder(med_pop_user_df_sample)
low_pop_user_df_sample = user_id_token_adder(low_pop_user_df_sample)
user_df_sample = user_id_token_adder(user_df_sample)
poi_df_sample = user_id_token_adder(poi_df_sample)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_58945/1404554777.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].astype(str) + "_x"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_58945/1404554777.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].astype(str) + "_x"


In [36]:
checkin_df_sample

,user_id:token,item_id:token,timestamp:float,checkin_count:float,business_popularity:float
1,0_x,0_x,Fri Apr 06 12:52:34 +0000 2012,2,0.061404
2,1_x,1_x,Fri Apr 06 12:52:35 +0000 2012,1,0.026316
3,2_x,2_x,Fri Apr 06 12:52:49 +0000 2012,33,0.031100
4,3_x,3_x,Fri Apr 06 12:53:38 +0000 2012,52,0.039872
5,4_x,4_x,Fri Apr 06 12:54:35 +0000 2012,11,0.009569
...,...,...,...,...,...
99680,190_x,1021_x,Wed Sep 05 23:15:07 +0000 2012,1,0.012759
99681,126_x,1877_x,Wed Sep 05 23:28:47 +0000 2012,1,0.015152
99682,1364_x,1051_x,Wed Sep 05 23:30:16 +0000 2012,2,0.018341
99683,100_x,471_x,Wed Sep 05 23:32:06 +0000 2012,1,0.028708


In [37]:
med_pop_user_df_sample

,user_id:token,average_popularity
2034,1129_x,0.142278
803,1491_x,0.142158
416,530_x,0.142158
1410,5_x,0.142045
2021,1443_x,0.141976
...,...,...
60,400_x,0.116345
1803,217_x,0.116162
516,625_x,0.116140
1404,1465_x,0.116065


In [38]:
# get a json with the user id's of the respective popularity groups
user_id_popularity = {}
user_id_popularity["high"] = high_pop_user_df_sample["user_id:token"].tolist()
user_id_popularity["medium"] = med_pop_user_df_sample["user_id:token"].tolist()
user_id_popularity["low"] = low_pop_user_df_sample["user_id:token"].tolist()
json.dump(user_id_popularity, open(f"{DATASET_DIR}/{dataset}_user_id_popularity.json", "w"))


In [39]:
def data_saver(df, filename, framework):
    if not os.path.exists(DATASET_DIR + "processed_data_" + framework):
        os.makedirs(DATASET_DIR + "processed_data_" + framework)
    
    df.to_csv(DATASET_DIR + "processed_data_" + framework + "/" + filename + ".csv")
    print("Data saved as " + framework + filename + ".csv")
    

In [40]:
# first of all saving data for cornac
data_saver(checkin_df_sample, "user_events", "cornac")
data_saver(high_pop_user_df_sample, "high_pop_user_sample", "cornac")
data_saver(med_pop_user_df_sample, "medium_pop_user_sample", "cornac")
data_saver(low_pop_user_df_sample, "low_pop_user_sample", "cornac")


Data saved as cornacuser_events.csv
Data saved as cornachigh_pop_user_sample.csv
Data saved as cornacmedium_pop_user_sample.csv
Data saved as cornaclow_pop_user_sample.csv


In [41]:
checkin_df_sample

,user_id:token,item_id:token,timestamp:float,checkin_count:float,business_popularity:float
1,0_x,0_x,Fri Apr 06 12:52:34 +0000 2012,2,0.061404
2,1_x,1_x,Fri Apr 06 12:52:35 +0000 2012,1,0.026316
3,2_x,2_x,Fri Apr 06 12:52:49 +0000 2012,33,0.031100
4,3_x,3_x,Fri Apr 06 12:53:38 +0000 2012,52,0.039872
5,4_x,4_x,Fri Apr 06 12:54:35 +0000 2012,11,0.009569
...,...,...,...,...,...
99680,190_x,1021_x,Wed Sep 05 23:15:07 +0000 2012,1,0.012759
99681,126_x,1877_x,Wed Sep 05 23:28:47 +0000 2012,1,0.015152
99682,1364_x,1051_x,Wed Sep 05 23:30:16 +0000 2012,2,0.018341
99683,100_x,471_x,Wed Sep 05 23:32:06 +0000 2012,1,0.028708


In [42]:
def data_saver_recbole(df, framework, suffix):
    
    if not os.path.exists(DATASET_DIR + "processed_data_" + framework):
        os.makedirs(DATASET_DIR + "processed_data_" + framework)

    df.to_csv(f"{DATASET_DIR}processed_data_{framework}/{dataset}_sample.{suffix}", sep="\t", index=False)

In [43]:
checkin_df_sample['review_id:token'] = range(1, len(checkin_df_sample) + 1)
# Step 1: Group by user_id and business_id and count check-ins
#checkin_df_sample['checkin_count:float'] = checkin_df_sample.groupby(['user_id:token', 'item_id:token'])['item_id:token'].transform('count')


In [44]:
checkin_df_timestamp

,user_id:token,item_id:token,timestamp:float
5689,0,0,Fri Apr 06 12:52:34 +0000 2012
5690,1,1,Fri Apr 06 12:52:35 +0000 2012
5692,2,2,Fri Apr 06 12:52:49 +0000 2012
5693,3,3,Fri Apr 06 12:53:38 +0000 2012
5694,4,4,Fri Apr 06 12:54:35 +0000 2012
...,...,...,...
331776,111,185,Wed Sep 05 23:55:47 +0000 2012
331782,379,88,Wed Sep 05 23:56:30 +0000 2012
331783,359,754,Wed Sep 05 23:57:37 +0000 2012
331784,403,148,Wed Sep 05 23:58:06 +0000 2012


In [45]:
checkin_df_sample = convert_to_unix_timestamp(checkin_df_sample, "timestamp:float")
checkin_df_timestamp = convert_to_unix_timestamp(checkin_df_timestamp, "timestamp:float")

In [46]:
checkin_df_timestamp

,user_id:token,item_id:token,timestamp:float
5689,0,0,1.333717e+09
5690,1,1,1.333717e+09
5692,2,2,1.333717e+09
5693,3,3,1.333717e+09
5694,4,4,1.333717e+09
...,...,...,...
331776,111,185,1.346889e+09
331782,379,88,1.346889e+09
331783,359,754,1.346889e+09
331784,403,148,1.346889e+09


In [47]:
checkin_df_sample

,user_id:token,item_id:token,timestamp:float,checkin_count:float,business_popularity:float,review_id:token
1,0_x,0_x,1.333717e+09,2,0.061404,1
2,1_x,1_x,1.333717e+09,1,0.026316,2
3,2_x,2_x,1.333717e+09,33,0.031100,3
4,3_x,3_x,1.333717e+09,52,0.039872,4
5,4_x,4_x,1.333717e+09,11,0.009569,5
...,...,...,...,...,...,...
99680,190_x,1021_x,1.346887e+09,1,0.012759,67522
99681,126_x,1877_x,1.346888e+09,1,0.015152,67523
99682,1364_x,1051_x,1.346888e+09,2,0.018341,67524
99683,100_x,471_x,1.346888e+09,1,0.028708,67525


In [48]:
checkin_df_sample.sort_values(by="checkin_count:float", ascending=False)
# very important: keeping the duplicate check-ins for the context aware recommendation to have the timestamps saved


# very important: dropping duplicate check-ins 
checkin_df_sample = checkin_df_sample.drop_duplicates(subset=["user_id:token", "item_id:token"], keep="first")

In [49]:
user_df_sample = user_df_sample[["user_id:token"]]

In [50]:
# This would be the correct splits if we let recbole do the splitting

# data_saver_recbole(checkin_df_sample, "recbole", "inter")
# data_saver_recbole(user_df_sample, "recbole", "user")
# data_saver_recbole(poi_df_sample, "recbole", "item")

In [51]:
poi_df_sample

,item_id:token,category_id:token,category_name:token_seq,lat:float,lon:float
7,156_x,4bf58dd8d48988d129951735,Train Station,35.700253,139.480255
10,431_x,4bf58dd8d48988d1eb931735,Airport,35.548963,139.784611
14,1031_x,4bf58dd8d48988d1df941735,Bridge,35.609929,139.825659
15,1428_x,4bf58dd8d48988d129951735,Train Station,35.749538,139.586540
17,184_x,4bf58dd8d48988d129951735,Train Station,35.729025,139.711096
...,...,...,...,...,...
412051,1509_x,4bf58dd8d48988d120941735,Bar,35.697700,139.770384
425285,1467_x,4bf58dd8d48988d16d941735,Café,35.702436,139.770470
450169,1440_x,4bf58dd8d48988d129951735,Train Station,35.607054,139.734894
462519,2582_x,4d954b0ea243a5684a65b473,Convenience Store,35.701178,139.771038


In [52]:
checkin_df_timestamp = checkin_df_timestamp[["user_id:token", "item_id:token", "timestamp:float"]] # FINAL
checkins_capri_train_test_tune = checkin_df_sample[["user_id:token", "item_id:token", "timestamp:float", "checkin_count:float"]]
try:
    poi_df_sample_capri = poi_df_sample[["item_id:token", "lat:float", "lon:float"]] # FINAL
except KeyError: # in the snowcard data the coordinates are not given
    poi_df_sample_capri = poi_df_sample[["item_id:token"]]
datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_train_test_tune["user_id:token"].unique())], "num_items" : [len(checkins_capri_train_test_tune["item_id:token"].unique())]}) # FINAL

### Creating train, test and val splits (user-based temporal split)

In [53]:
# splitting the data into train, test, and tune
checkins_capri_train_test_tune = checkins_capri_train_test_tune.sort_values(by=["user_id:token", "timestamp:float"])
checkins_capri_train_test_tune = checkins_capri_train_test_tune[["user_id:token", "item_id:token", "checkin_count:float"]]

# Split the data
train_list = []
val_list = []
test_list = []

for user, group in checkins_capri_train_test_tune.groupby('user_id:token'):
    n = len(group)
    train_end = int(n * 0.65)
    val_end = int(n * 0.80)
    
    train_list.append(group.iloc[:train_end])
    val_list.append(group.iloc[train_end:val_end])
    test_list.append(group.iloc[val_end:])

# Combine lists into DataFrames
train_df = pd.concat(train_list)
val_df = pd.concat(val_list)
test_df = pd.concat(test_list)



# Check the splits

# FINAL 6-8
print("Train Set:")
print(train_df.head())
print("\nValidation Set:")
print(val_df.head())
print("\nTest Set:")
print(test_df.head())


Train Set:
      user_id:token item_id:token  checkin_count:float
1               0_x           0_x                    2
150             0_x          87_x                    1
152             0_x          89_x                   15
154             0_x          91_x                   13
59655           0_x         889_x                    1

Validation Set:
      user_id:token item_id:token  checkin_count:float
72716           0_x         126_x                    1
72718           0_x         745_x                    2
72828           0_x        1957_x                    1
18048           0_x         171_x                    2
27005        1000_x        1997_x                    1

Test Set:
      user_id:token item_id:token  checkin_count:float
83351           0_x          68_x                    1
73208           0_x         193_x                    1
73209           0_x         213_x                    1
73803           0_x         831_x                    1
73871           0_x       

In [54]:
def datasaver_capri(df, filename):
    
    if not os.path.exists(DATASET_DIR + "processed_data_capri"):
        os.makedirs(DATASET_DIR + "processed_data_capri")
    
    df.to_csv(DATASET_DIR + "processed_data_capri/" + filename + ".txt", sep='\t', index=False, header=False)
    print("Data saved as " + filename + ".txt")
    



In [9]:
# adding a category column
if include_categories is True:
    if dataset == "yelp":
        # Split the 'category_name' column by commas
        poi_df_sample['category_name_unstacked:token_seq'] = poi_df_sample['category_name:token_seq'].str.split(', ')

        # Unstack the categories into multiple rows
        category_df_sample = poi_df_sample.explode('category_name_unstacked:token_seq')
        category_counts = category_df_sample["category_name_unstacked:token_seq"].value_counts()
        category_mask = category_df_sample["category_name_unstacked:token_seq"].map(category_counts) >= 25
        category_df_sample_filtered = category_df_sample.loc[category_mask]
        category_df_sample_filtered["category_id:token"], category_id = pd.factorize(category_df_sample_filtered["category_name_unstacked:token_seq"])
        category_df_sample_filtered.dropna(inplace=True)
        datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_train_test_tune["user_id:token"].unique())], "num_items" : [len(checkins_capri_train_test_tune["item_id:token"].unique())], "num_categories" : [len(category_df_sample_filtered["category_id:token"].unique())]}) # FINAL
        datasaver_capri(category_df_sample_filtered, "poiCategories")


    elif dataset == "foursquarenyc" or dataset == "foursquaretky":
        poi_df_sample["category_id:token"], category_id = pd.factorize(poi_df_sample["category_name:token_seq"])
        datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_train_test_tune["user_id:token"].unique())], "num_items" : [len(checkins_capri_train_test_tune["item_id:token"].unique())], "num_categories" : [len(poi_df_sample["category_id:token"].unique())]})
        poi_df_categories = poi_df_sample[["item_id:token", "category_id:token"]]
        datasaver_capri(poi_df_categories, "poiCategories")

    elif dataset == "snowcard":
        datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_train_test_tune["user_id:token"].unique())], "num_items" : [len(checkins_capri_train_test_tune["item_id:token"].unique())], "num_categories" : [len(poi_df_sample["category_id:token"].unique())]})
        poi_df_categories = poi_df_sample[["item_id:token", "category_id:token"]]
        datasaver_capri(poi_df_categories, "poiCategories")
    

In [56]:
train_df["user_id:token"].nunique(), val_df["user_id:token"].nunique(), test_df["user_id:token"].nunique()

(1500, 1500, 1500)

In [57]:
# This is the correct split since we perform the splitting ourselves

data_saver_recbole(train_df, "recbole", "train.inter")
data_saver_recbole(test_df, "recbole", "test.inter")
data_saver_recbole(val_df, "recbole", "valid.inter")

### Make an intervened sample for RecBole debias

In [58]:
checkins_debias = checkins_capri_train_test_tune.copy()
intervened_set = checkins_debias.sample(frac=0.5, random_state=10002)
normal_set = checkins_debias.drop(intervened_set.index)

original_user_ids = set(checkins_debias["user_id:token"].unique())
intervened_user_ids = set(intervened_set["user_id:token"].unique())
normal_user_ids = set(normal_set["user_id:token"].unique())

missing_in_intervened = original_user_ids - intervened_user_ids
missing_in_normal = original_user_ids - normal_user_ids

if not missing_in_intervened and not missing_in_normal:
    print("All user IDs are present in both subsets.")
else:
    print("Missing user IDs in intervened set:", missing_in_intervened)
    print("Missing user IDs in normal set:", missing_in_normal)

All user IDs are present in both subsets.


In [59]:
# SOURCE: https://github.com/DavyMorgan/dps/blob/19a3e5fb2cb6932c3f093ed443760ecd3d95bfdb/data_process_service/splitter.py

popularity = (
            intervened_set[["item_id:token", "user_id:token"]]
            .groupby("item_id:token")
            .count()
            .reset_index()
            .rename(columns={"user_id:token": "pop"})
        )
intervened_set = intervened_set.merge(popularity, on="item_id:token", how="left")
intervened_set["pop"] = intervened_set["pop"].apply(lambda x: 1 / x)

In [60]:
intervened_set.sort_values(by=["pop", "item_id:token"], ascending=False, inplace=True)
intervened_set.drop(columns=["pop"], inplace=True)

In [61]:
# Lists to collect data for each split
train_list_intervened, val_list_intervened, test_list_intervened = [], [], []

# Loop through each user's data in the sorted intervened set
for user, group in intervened_set.groupby('user_id:token'):
    n = len(group)
    
    # Calculate end indices based on the desired split ratios
    test_end = int(n * 0.5)  # Top 50% for test
    train_end = test_end + int(n * 0.25)  # Next 25% for train
    
    # Add to respective lists
    test_list_intervened.append(group.iloc[:test_end])  # First 50% goes to test
    train_list_intervened.append(group.iloc[test_end:train_end])  # Next 25% goes to train
    val_list_intervened.append(group.iloc[train_end:])  # Remaining 25% goes to validation

# Concatenate lists into DataFrames
train_df_intervened = pd.concat(train_list_intervened, ignore_index=True)
train_df_intervened = pd.concat([train_df_intervened, normal_set], ignore_index=True).sort_values(by=["user_id:token"])
val_df_intervened = pd.concat(val_list_intervened, ignore_index=True)
test_df_intervened = pd.concat(test_list_intervened, ignore_index=True)

# Output the sizes of each set
print("Intervened Training Set Size:", len(train_df_intervened))
print("Intervened Validation Set Size:", len(val_df_intervened))
print("Intervened Test Set Size:", len(test_df_intervened))


Intervened Training Set Size: 41622
Intervened Validation Set Size: 9409
Intervened Test Set Size: 16495


In [62]:
train_df_intervened

,user_id:token,item_id:token,checkin_count:float
0,0_x,775_x,1
7861,0_x,89_x,15
7862,0_x,7_x,5
7863,0_x,31_x,1
7864,0_x,50_x,2
...,...,...,...
41590,9_x,674_x,15
41591,9_x,146_x,1
41592,9_x,2459_x,1
41594,9_x,433_x,2


In [63]:
data_saver_recbole(train_df_intervened, "recbole_debias", "train.inter")
data_saver_recbole(test_df_intervened, "recbole_debias", "test.inter")
data_saver_recbole(val_df_intervened, "recbole_debias", "valid.inter")

### Save for CAPRI without _x since they require integers as IDs

In [64]:
def user_id_cleaner(df, column_name_list = ["user_id:token", "item_id:token"]):
    for column_name in column_name_list:
        df[column_name] = df[column_name].str.split("_")
        df[column_name] = df[column_name].apply(lambda x: x[0])

    return df

In [65]:
poi_df_sample_capri = user_id_cleaner(poi_df_sample_capri, ["item_id:token"])
train_df = user_id_cleaner(train_df)
val_df = user_id_cleaner(val_df)
test_df = user_id_cleaner(test_df)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_58945/2072793301.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].str.split("_")
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_58945/2072793301.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(lambda x: x[0])


In [66]:
datasaver_capri(checkin_df_timestamp, "checkins")
datasaver_capri(datasize_capri, "dataSize")
datasaver_capri(poi_df_sample_capri, "poiCoos")
datasaver_capri(train_df, "train")
datasaver_capri(val_df, "tune")
datasaver_capri(test_df, "test")

Data saved as checkins.txt
Data saved as dataSize.txt
Data saved as poiCoos.txt
Data saved as train.txt
Data saved as tune.txt
Data saved as test.txt


In [67]:
train_df["item_id:token"].nunique()

train_df["user_id:token"] = train_df["user_id:token"].astype(int)

In [68]:
train_df["item_id:token"].nunique()


2799

In [69]:

train_df.sort_values(by="item_id:token", ascending=False)

,user_id:token,item_id:token,checkin_count:float
30184,158,999,2
23997,316,999,1
59406,32,999,1
33937,1484,999,1
90836,349,999,1
...,...,...,...
76823,285,0,1
31958,496,0,2
29880,362,0,1
73601,485,0,1
